In [ ]:
!pip install webdriver-manager

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import urllib.parse
import requests
import pandas as pd
import warnings
import re
import csv
import time
import random

warnings.filterwarnings("ignore")


# Ignore any warnings
warnings.filterwarnings("ignore")

# Define the login credentials
print("Please enter your LinkedIn credentials:")
print("Email: ")
username = input()
print("Password: ")
password = input()

# Initialize the Chrome driver
service = Service(executable_path=r"C:\Users\snehi\Desktop\Linkedin_extract\chromedriver.exe") # latest 132v chrome driver
driver = webdriver.Chrome(service=service)
#driver = webdriver.Chrome(executable_path=r"C:\Users\snehi\Desktop\Linkedin_extract\chromedriver.exe") 
driver.maximize_window()

# Navigate to the LinkedIn login page
driver.get("https://www.linkedin.com/login")

# Find the input fields and enter the login credentials
username_field = driver.find_element(By.ID, "username")
username_field.send_keys(username)

password_field = driver.find_element(By.ID, "password")
password_field.send_keys(password)

# Submit the login form
password_field.send_keys(Keys.RETURN)

# Wait for the page to load after login
time.sleep(5)

# Wait for the page to load after login
time.sleep(5)


user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    # Add more user-agents
]
headers = {'User-Agent': random.choice(user_agents)}


print("LinkedIn LOGGED IN Successfully..................START Extracting the postings")


### CUSTOM RUNNING VERSION (just extracts linkedin job posts, but NOT External LINKS )
### This below script extracts the job postings from specific LinkedIn search results page with a given post date

urllink = 'https://www.linkedin.com/search/results/content/?datePosted=%22past-24h%22&keywords=(%22Data%20Analyst%22%20OR%20%22Business%20Analyst%22)%20AND%20(%22Hiring%22%20OR%20%22Looking%20for%22)&origin=GLOBAL_SEARCH_HEADER&sid=%40%2CL&sortBy=%22relevance%22'
post_date = "2025-01-01"  # User input for the post date

driver.get(urllink)


# Wait for page to load
time.sleep(5)


# Function to extract job postings
def extract_postings(soup):
    extract_posting = soup.find_all('div', class_='fie-impression-container')
    postings = []
    for post in extract_posting:
        job_title_element = post.find('h2', class_='update-components-entity__title')
        if job_title_element:
            job_title = job_title_element.text.strip()
        else:
            job_title = ''
        
        location_element = post.find('h3', class_='update-components-entity__description')
        if location_element:
            location = location_element.text.strip()
            location_parts = location.split(', ')
            city = location_parts[0]
            state = location_parts[1] if len(location_parts) > 1 else ''
            country = location_parts[2] if len(location_parts) > 2 else ''
        else:
            city = state = country = ''
        
        company_element = post.find('h3', class_='update-components-entity__subtitle')
        if company_element:
            company = company_element.text.strip().replace('Job by ', '')
            company_link = f"https://www.linkedin.com/company/{company.replace(' ', '-')}/"
        else:
            company = ''
        
        post_hour_text_element = post.find('a', class_='app-aware-link update-components-actor__sub-description-link')
        if post_hour_text_element:
            post_hour_text = post_hour_text_element.text.strip()
            post_hour = re.search(r'\d+', post_hour_text).group()
            post_hour = int(post_hour)
        else:
            post_hour = ''
        
        hiring_employee_name_element = post.find('span', class_='update-components-actor__name')
        if hiring_employee_name_element:
            hiring_employee_name = hiring_employee_name_element.text.strip()
        else:
            hiring_employee_name = ''
        
        hiring_employee_profile_link_element = post.find('a', class_='app-aware-link update-components-actor__image relative')
        if hiring_employee_profile_link_element:
            hiring_employee_profile_link = hiring_employee_profile_link_element['href']
        else:
            hiring_employee_profile_link = ''
        
        job_link_element = post.find('a', class_='app-aware-link update-components-entity__content display-flex')
        if job_link_element:
            job_link = job_link_element['href']
        else:
            # Check for the new format 
            job_link_element_new = post.find('a', class_='app-aware-link  update-components-article__meta flex-grow-1 full-width tap-target display-flex justify-space-between align-items-flex-start') 
            if job_link_element_new: 
                job_link = job_link_element_new['href'] 
            else: 
                job_link = ''

        posting = {
            'title': job_title,
            'city': city,
            'state': state,
            'country': country,
            'company_link': company_link,
            'company': company,
            'Post hour': post_hour,
            'hiring/employee_name': hiring_employee_name,
            'hiring/employee_profile_link':  hiring_employee_profile_link,
            'post_date': post_date,
            'Job_link':  job_link
        }
        
        print(posting)
        # Only add posting if job_link is available
        if job_link:
            postings.append(posting)
    
    return postings


# Initialize CSV writer
import csv
with open('linkedin_hiringposts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['title', 'city', 'state', 'country', 'company_link', 'company', 'Post hour', 'hiring/employee_name', 'hiring/employee_profile_link', 'post_date', 'Job_link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()


# Initialize postings list
postings = []


# Initial extraction
soup = BeautifulSoup(driver.page_source, 'html.parser')
new_postings = extract_postings(soup)
for posting in new_postings:
    if posting not in postings:
        postings.append(posting)


# Scroll and extract more postings
last_height = driver.execute_script('return document.body.scrollHeight')
scroll_count = 0
max_scrolls = 10


while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)
    new_height = driver.execute_script('return document.body.scrollHeight')
    scroll_count += 1


    if new_height == last_height or scroll_count >= max_scrolls:
        break


    soup = BeautifulSoup(driver.page_source, 'html.parser')
    new_postings = extract_postings(soup)
    for posting in new_postings:
        if posting not in postings:
            postings.append(posting)
    last_height = new_height


# Write postings to CSV
with open('linkedin_hiringposts.csv', 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['title', 'city', 'state', 'country', 'company_link', 'company', 'Post hour', 'hiring/employee_name', 'hiring/employee_profile_link', 'post_date', 'Job_link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for posting in postings:
        writer.writerow(posting)




print("done")